# Day13_1: 분류 모델 (Classification Models) - 정답

---

In [ ]:
# 공통 라이브러리 임포트
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    classification_report, confusion_matrix,
    roc_curve, auc, precision_recall_curve, average_precision_score
)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import (
    RandomForestClassifier, GradientBoostingClassifier,
    VotingClassifier, BaggingClassifier, AdaBoostClassifier,
    StackingClassifier
)

import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료!")

---

## Q1. Logistic Regression 기본 훈련 ⭐

**문제**: 아래 데이터로 Logistic Regression 모델을 훈련하고 정확도를 출력하세요.

In [ ]:
# 정답 코드
from sklearn.datasets import load_breast_cancer
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

# 데이터 로드
data = load_breast_cancer()
X, y = data.data, data.target

# 훈련/테스트 분할
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

# 모델 훈련
lr_model = LogisticRegression(random_state=42, max_iter=10000)
lr_model.fit(X_train, y_train)

# 정확도 출력
accuracy = lr_model.score(X_test, y_test)
print(f"테스트 정확도: {accuracy:.4f}")

In [ ]:
# 테스트
assert accuracy > 0.9, "정확도가 90% 이상이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. breast_cancer 데이터셋 로드 (이진 분류: 악성/양성)
2. train_test_split으로 80:20 분할
3. LogisticRegression 모델 훈련
4. score() 메서드로 정확도 계산

**핵심 개념**:
- `load_breast_cancer()`: 569개 샘플, 30개 특성의 이진 분류 데이터
- `max_iter=10000`: 수렴을 위한 충분한 반복 횟수
- `random_state=42`: 재현성 보장

**대안**:
- `accuracy_score(y_test, lr_model.predict(X_test))`로 동일 결과
- 스케일링 적용 시 더 안정적인 결과

**흔한 실수**:
- max_iter 미설정으로 수렴 경고 발생
- 데이터 분할 없이 전체 데이터로 평가 (과적합)

**실무 팁**:
- Logistic Regression은 베이스라인 모델로 항상 먼저 시도
- 계수(coef_)로 특성 중요도 해석 가능

---

## Q2. KNN 모델 K값 실험 ⭐

**문제**: K=3, K=5, K=7로 KNN 모델을 훈련하고 각각의 정확도를 비교하세요.

In [ ]:
# 정답 코드
from sklearn.neighbors import KNeighborsClassifier

# 데이터 준비 (스케일링 적용)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# K값 실험
k_values = [3, 5, 7]
results = []

for k in k_values:
    # 모델 생성 및 훈련
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    # 정확도 계산
    accuracy = knn.score(X_test_scaled, y_test)
    results.append({'K': k, 'Accuracy': accuracy})
    print(f"K={k}: 정확도 {accuracy:.4f}")

# 결과 정리
df_knn_results = pd.DataFrame(results)
best_k = df_knn_results.loc[df_knn_results['Accuracy'].idxmax(), 'K']
print(f"\n최적 K: {best_k}")

In [ ]:
# 테스트
assert len(results) == 3, "3개의 K값에 대한 결과가 있어야 합니다"
assert all(r['Accuracy'] > 0.9 for r in results), "모든 정확도가 90% 이상이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. KNN은 거리 기반이므로 스케일링 필수
2. 여러 K값으로 반복 실험
3. 최적 K값 선택

**핵심 개념**:
- K가 작으면 과적합, K가 크면 과소적합
- 일반적으로 홀수 K 사용 (동점 방지)
- StandardScaler로 특성 스케일 통일

**대안**:
- cross_val_score로 더 안정적인 K값 선택
- GridSearchCV로 자동 최적화

**흔한 실수**:
- 스케일링 없이 KNN 적용 (성능 저하)
- 테스트 데이터에도 fit_transform 적용 (데이터 누수)

**실무 팁**:
- KNN은 소규모 데이터에 적합
- 대규모 데이터는 예측 시간이 오래 걸림

---

## Q3. Decision Tree 특성 중요도 추출 ⭐⭐

**문제**: Decision Tree를 훈련하고 상위 5개 중요 특성을 출력하세요.

In [ ]:
# 정답 코드
from sklearn.tree import DecisionTreeClassifier

# 특성 이름
feature_names = data.feature_names

# Decision Tree 훈련
dt_model = DecisionTreeClassifier(max_depth=5, random_state=42)
dt_model.fit(X_train, y_train)

# 특성 중요도 추출
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': dt_model.feature_importances_
}).sort_values('importance', ascending=False)

# 상위 5개 출력
print("상위 5개 중요 특성:")
print(importance_df.head(5).to_string(index=False))

# 시각화
top5 = importance_df.head(5)
fig = go.Figure(go.Bar(
    x=top5['importance'],
    y=top5['feature'],
    orientation='h',
    marker_color='steelblue'
))
fig.update_layout(
    title="Decision Tree 특성 중요도 (Top 5)",
    xaxis_title="Importance",
    yaxis=dict(autorange='reversed'),
    template="plotly_white"
)
fig.show()

In [ ]:
# 테스트
assert len(importance_df) == 30, "30개 특성의 중요도가 있어야 합니다"
assert importance_df['importance'].sum() > 0.99, "중요도 합이 1에 가까워야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. Decision Tree 훈련 (max_depth로 과적합 방지)
2. feature_importances_ 속성으로 중요도 추출
3. DataFrame으로 정리 후 정렬

**핵심 개념**:
- feature_importances_: 각 특성이 분류에 기여하는 정도
- 불순도 감소량 기반 계산 (Gini impurity)
- 합이 1이 되도록 정규화됨

**대안**:
- Random Forest의 feature_importances_가 더 안정적
- permutation_importance()로 더 정확한 중요도 계산

**흔한 실수**:
- max_depth 미설정으로 과적합
- 특성 이름과 중요도 매칭 오류

**실무 팁**:
- 특성 중요도는 특성 선택(feature selection)에 활용
- 중요도가 0인 특성은 제거 고려

---

## Q4. Random Forest vs Decision Tree 비교 ⭐⭐

**문제**: Random Forest (n_estimators=100)와 Decision Tree의 성능을 비교하세요.

In [ ]:
# 정답 코드
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score

# Decision Tree
dt = DecisionTreeClassifier(max_depth=5, random_state=42)
dt.fit(X_train, y_train)
y_pred_dt = dt.predict(X_test)

# Random Forest
rf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=42, n_jobs=-1)
rf.fit(X_train, y_train)
y_pred_rf = rf.predict(X_test)

# 성능 비교
results = {
    'Model': ['Decision Tree', 'Random Forest'],
    'Accuracy': [accuracy_score(y_test, y_pred_dt), accuracy_score(y_test, y_pred_rf)],
    'F1 Score': [f1_score(y_test, y_pred_dt), f1_score(y_test, y_pred_rf)]
}

df_comparison = pd.DataFrame(results)
print("성능 비교:")
print(df_comparison.to_string(index=False))

# 승자 판정
winner = 'Random Forest' if results['F1 Score'][1] > results['F1 Score'][0] else 'Decision Tree'
print(f"\n승자: {winner}")

In [ ]:
# 테스트
assert results['Accuracy'][1] >= results['Accuracy'][0] * 0.95, "RF가 DT보다 크게 나쁘면 안됩니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 동일 조건(max_depth)으로 두 모델 훈련
2. 동일 지표(Accuracy, F1)로 비교
3. 결과 테이블로 정리

**핵심 개념**:
- Random Forest = Decision Tree의 앙상블
- 배깅으로 분산 감소, 과적합 방지
- n_estimators: 트리 개수 (많을수록 안정적)

**대안**:
- cross_val_score로 더 신뢰성 있는 비교
- 학습 시간, 예측 시간도 비교 가능

**흔한 실수**:
- 다른 조건으로 비교 (불공정)
- 단일 지표만으로 판단

**실무 팁**:
- Random Forest는 거의 항상 Decision Tree보다 좋음
- 해석이 중요하면 Decision Tree 선택

---

## Q5. Confusion Matrix 계산 ⭐⭐

**문제**: 사기 탐지 데이터에서 Random Forest의 Confusion Matrix를 출력하고 TP, FP, TN, FN을 해석하세요.

In [ ]:
# 사기 탐지 데이터 생성
np.random.seed(42)

# 정상 거래 (95%)
n_normal = 9500
normal_data = {
    'amount': np.random.exponential(scale=100, size=n_normal),
    'hour': np.random.randint(6, 23, n_normal),
    'distance_from_home': np.random.exponential(scale=10, size=n_normal),
    'ratio_to_median': np.random.uniform(0.5, 2.0, n_normal),
    'fraud': np.zeros(n_normal)
}

# 사기 거래 (5%)
n_fraud = 500
fraud_data = {
    'amount': np.random.exponential(scale=500, size=n_fraud),
    'hour': np.random.choice([0, 1, 2, 3, 4, 5, 23], n_fraud),
    'distance_from_home': np.random.exponential(scale=100, size=n_fraud),
    'ratio_to_median': np.random.uniform(3.0, 10.0, n_fraud),
    'fraud': np.ones(n_fraud)
}

df_fraud = pd.concat([pd.DataFrame(normal_data), pd.DataFrame(fraud_data)], ignore_index=True)
df_fraud = df_fraud.sample(frac=1, random_state=42).reset_index(drop=True)

X_fraud = df_fraud.drop('fraud', axis=1)
y_fraud = df_fraud['fraud']

X_train_f, X_test_f, y_train_f, y_test_f = train_test_split(
    X_fraud, y_fraud, test_size=0.2, random_state=42, stratify=y_fraud
)

In [ ]:
# 정답 코드
from sklearn.metrics import confusion_matrix

# Random Forest 훈련
rf_fraud = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_fraud.fit(X_train_f, y_train_f)
y_pred_fraud = rf_fraud.predict(X_test_f)

# Confusion Matrix 계산
cm = confusion_matrix(y_test_f, y_pred_fraud)
TN, FP, FN, TP = cm.ravel()

print("Confusion Matrix:")
print(cm)
print(f"\nTN (True Negative): {TN}")
print(f"  -> 정상 거래를 정상으로 올바르게 예측")
print(f"FP (False Positive): {FP}")
print(f"  -> 정상 거래를 사기로 잘못 예측 (허위 경보)")
print(f"FN (False Negative): {FN}")
print(f"  -> 사기 거래를 정상으로 잘못 예측 (놓친 사기!)")
print(f"TP (True Positive): {TP}")
print(f"  -> 사기 거래를 사기로 올바르게 예측")

# 비용 관점 해석
print(f"\n[비용 관점 해석]")
print(f"FN({FN}건)이 가장 위험: 실제 사기를 놓치면 금전적 손실 발생")
print(f"FP({FP}건)는 고객 불편: 정상 거래 차단으로 고객 이탈 우려")

In [ ]:
# 테스트
assert TN + FP + FN + TP == len(y_test_f), "CM 합이 테스트 데이터 수와 같아야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. confusion_matrix()로 CM 계산
2. ravel()로 TN, FP, FN, TP 추출
3. 각 값의 의미를 비즈니스 관점에서 해석

**핵심 개념**:
- TN: 실제 음성을 음성으로 예측 (정상->정상)
- FP: 실제 음성을 양성으로 예측 (정상->사기)
- FN: 실제 양성을 음성으로 예측 (사기->정상) **위험!**
- TP: 실제 양성을 양성으로 예측 (사기->사기)

**대안**:
- ConfusionMatrixDisplay로 시각화
- Plotly로 히트맵 시각화

**흔한 실수**:
- TN/FP/FN/TP 순서 혼동 (sklearn은 [[TN,FP],[FN,TP]])
- 양성/음성 기준 혼동

**실무 팁**:
- 사기 탐지에서 FN 비용 >> FP 비용
- Recall을 높이는 것이 중요 (FN 최소화)

---

## Q6. Voting Classifier 구현 ⭐⭐⭐

**문제**: LogisticRegression, RandomForest, GradientBoosting을 결합한 Soft Voting Classifier를 구현하세요.

In [ ]:
# 정답 코드
from sklearn.ensemble import VotingClassifier, GradientBoostingClassifier

# 개별 모델 정의
lr = LogisticRegression(random_state=42, max_iter=1000)
rf = RandomForestClassifier(n_estimators=50, random_state=42, n_jobs=-1)
gb = GradientBoostingClassifier(n_estimators=50, random_state=42)

# Voting Classifier (Soft Voting)
voting_clf = VotingClassifier(
    estimators=[
        ('lr', lr),
        ('rf', rf),
        ('gb', gb)
    ],
    voting='soft'  # 확률 평균
)

# 개별 모델 훈련 및 평가
models = {'Logistic Regression': lr, 'Random Forest': rf, 'Gradient Boosting': gb}
individual_scores = {}

for name, model in models.items():
    model.fit(X_train_f, y_train_f)
    score = model.score(X_test_f, y_test_f)
    individual_scores[name] = score
    print(f"{name}: {score:.4f}")

# Voting Classifier 훈련 및 평가
voting_clf.fit(X_train_f, y_train_f)
voting_score = voting_clf.score(X_test_f, y_test_f)
print(f"\nVoting Classifier: {voting_score:.4f}")

# 비교
best_individual = max(individual_scores.values())
improvement = voting_score - best_individual
print(f"\n앙상블 효과: {improvement:+.4f}")

In [ ]:
# 테스트
assert voting_score > 0.9, "Voting 정확도가 90% 이상이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 서로 다른 특성의 모델 3개 선택 (선형, 트리, 부스팅)
2. VotingClassifier로 결합
3. soft voting으로 확률 평균 사용

**핵심 개념**:
- Hard Voting: 다수결 (예측 클래스 투표)
- Soft Voting: 확률 평균 (일반적으로 더 좋음)
- 다양한 모델 결합이 성능 향상에 유리

**대안**:
- weights 파라미터로 모델별 가중치 조정
- 더 다양한 모델 추가 (SVM, KNN 등)

**흔한 실수**:
- Hard voting에서 probability=True 미설정
- 비슷한 모델만 결합 (다양성 부족)

**실무 팁**:
- Voting은 간단하면서 효과적인 앙상블
- 모델 다양성이 핵심

---

## Q7. class_weight 효과 분석 ⭐⭐⭐

**문제**: 사기 탐지 데이터에서 class_weight 적용 전후의 Recall을 비교하세요.

In [ ]:
# 정답 코드
from sklearn.metrics import recall_score

# 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_f)
X_test_scaled = scaler.transform(X_test_f)

# class_weight 없음
lr_no_weight = LogisticRegression(random_state=42, max_iter=1000)
lr_no_weight.fit(X_train_scaled, y_train_f)
y_pred_no_weight = lr_no_weight.predict(X_test_scaled)
recall_no_weight = recall_score(y_test_f, y_pred_no_weight)

# class_weight='balanced'
lr_balanced = LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000)
lr_balanced.fit(X_train_scaled, y_train_f)
y_pred_balanced = lr_balanced.predict(X_test_scaled)
recall_balanced = recall_score(y_test_f, y_pred_balanced)

# 비교
print("Recall 비교 (사기 클래스):")
print(f"  class_weight 없음: {recall_no_weight:.4f}")
print(f"  class_weight='balanced': {recall_balanced:.4f}")
print(f"\n개선: {recall_balanced - recall_no_weight:+.4f} ({(recall_balanced/recall_no_weight - 1)*100:+.1f}%)")

# Precision도 확인
precision_no_weight = precision_score(y_test_f, y_pred_no_weight)
precision_balanced = precision_score(y_test_f, y_pred_balanced)
print(f"\n[참고] Precision:")
print(f"  class_weight 없음: {precision_no_weight:.4f}")
print(f"  class_weight='balanced': {precision_balanced:.4f}")
print(f"\n-> Recall 증가, Precision 감소 (트레이드오프)")

In [ ]:
# 테스트
assert recall_balanced >= recall_no_weight, "balanced가 Recall이 더 높거나 같아야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. class_weight 없이 기본 모델 훈련
2. class_weight='balanced'로 모델 훈련
3. Recall과 Precision 비교

**핵심 개념**:
- class_weight='balanced': n_samples / (n_classes * np.bincount(y))
- 소수 클래스에 높은 가중치 -> 오분류 페널티 증가
- Recall 증가, Precision 감소 (트레이드오프)

**대안**:
- class_weight={0: 1, 1: 10}으로 수동 설정
- SMOTE로 오버샘플링

**흔한 실수**:
- Recall만 보고 Precision 무시
- 모든 모델에 class_weight 적용 가능하다고 가정

**실무 팁**:
- 사기 탐지: Recall 중시 (FN 비용 높음)
- 스팸 필터: Precision 중시 (FP 비용 높음)

---

## Q8. ROC 곡선 Plotly 시각화 ⭐⭐⭐

**문제**: Logistic Regression과 Random Forest의 ROC 곡선을 Plotly로 시각화하세요.

In [ ]:
# 정답 코드
import plotly.graph_objects as go
from sklearn.metrics import roc_curve, auc

# 모델 준비 (앞서 훈련한 모델 사용)
# LR 확률 예측
y_proba_lr = lr_balanced.predict_proba(X_test_scaled)[:, 1]

# RF 확률 예측
rf_model = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
rf_model.fit(X_train_f, y_train_f)
y_proba_rf = rf_model.predict_proba(X_test_f)[:, 1]

# ROC 곡선 계산
fpr_lr, tpr_lr, _ = roc_curve(y_test_f, y_proba_lr)
auc_lr = auc(fpr_lr, tpr_lr)

fpr_rf, tpr_rf, _ = roc_curve(y_test_f, y_proba_rf)
auc_rf = auc(fpr_rf, tpr_rf)

# Plotly 시각화
fig = go.Figure()

# Logistic Regression
fig.add_trace(go.Scatter(
    x=fpr_lr, y=tpr_lr,
    mode='lines',
    name=f'Logistic Regression (AUC={auc_lr:.3f})',
    line=dict(color='blue', width=2)
))

# Random Forest
fig.add_trace(go.Scatter(
    x=fpr_rf, y=tpr_rf,
    mode='lines',
    name=f'Random Forest (AUC={auc_rf:.3f})',
    line=dict(color='green', width=2)
))

# 대각선 (랜덤 분류기)
fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random (AUC=0.5)',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title='ROC Curve: LR vs RF',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_white',
    legend=dict(x=0.6, y=0.1)
)
fig.show()

print(f"\nAUC 비교:")
print(f"  Logistic Regression: {auc_lr:.4f}")
print(f"  Random Forest: {auc_rf:.4f}")

In [ ]:
# 테스트
assert auc_lr > 0.5, "LR AUC가 0.5보다 커야 합니다"
assert auc_rf > 0.5, "RF AUC가 0.5보다 커야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. predict_proba()로 확률 예측
2. roc_curve()로 FPR, TPR 계산
3. auc()로 AUC 계산
4. Plotly로 시각화

**핵심 개념**:
- ROC: 임계값 변화에 따른 TPR vs FPR
- AUC: ROC 곡선 아래 면적 (1에 가까울수록 좋음)
- 대각선(AUC=0.5)은 랜덤 분류기 기준선

**대안**:
- sklearn의 RocCurveDisplay 사용
- matplotlib으로 시각화

**흔한 실수**:
- predict() 대신 predict_proba() 사용해야 함
- 클래스 1의 확률만 사용 ([:, 1])

**실무 팁**:
- AUC는 임계값에 독립적인 평가 지표
- 불균형 데이터에서는 PR 곡선이 더 유용

---

## Q9. Stacking Ensemble 구현 ⭐⭐⭐⭐

**문제**: 3개의 기본 모델과 메타 모델을 사용한 Stacking Classifier를 구현하세요.

In [ ]:
# 정답 코드
from sklearn.ensemble import StackingClassifier
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 기본 모델 정의
base_models = [
    ('knn', KNeighborsClassifier(n_neighbors=5)),
    ('dt', DecisionTreeClassifier(max_depth=5, random_state=42)),
    ('svm', SVC(probability=True, random_state=42))
]

# 메타 모델
meta_model = LogisticRegression(random_state=42, max_iter=1000)

# Stacking Classifier
stacking_clf = StackingClassifier(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5  # 교차 검증으로 메타 특성 생성
)

# 훈련 (스케일링된 데이터 사용)
stacking_clf.fit(X_train_scaled, y_train_f)

# 예측
y_pred_stacking = stacking_clf.predict(X_test_scaled)

# 평가
print("Stacking Classifier 결과:")
print(f"정확도: {accuracy_score(y_test_f, y_pred_stacking):.4f}")
print("\n" + classification_report(y_test_f, y_pred_stacking, target_names=['정상', '사기']))

In [ ]:
# 개별 모델과 비교
print("\n개별 모델 vs Stacking 비교:")
print("="*40)

individual_f1 = []
for name, model in base_models:
    model.fit(X_train_scaled, y_train_f)
    y_pred = model.predict(X_test_scaled)
    f1 = f1_score(y_test_f, y_pred)
    individual_f1.append(f1)
    print(f"{name}: F1={f1:.4f}")

stacking_f1 = f1_score(y_test_f, y_pred_stacking)
print(f"\nStacking: F1={stacking_f1:.4f}")
print(f"최고 개별 모델 대비: {stacking_f1 - max(individual_f1):+.4f}")

In [ ]:
# 테스트
assert accuracy_score(y_test_f, y_pred_stacking) > 0.9, "Stacking 정확도가 90% 이상이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 서로 다른 특성의 기본 모델 3개 정의
2. 메타 모델로 Logistic Regression 사용
3. StackingClassifier로 결합
4. cv=5로 교차 검증 기반 메타 특성 생성

**핵심 개념**:
- Stacking: 기본 모델의 예측을 새로운 특성으로 사용
- 메타 모델이 최종 예측 수행
- cv로 데이터 누수 방지

**대안**:
- 메타 모델로 더 강력한 모델 사용 (GradientBoosting)
- passthrough=True로 원본 특성도 포함

**흔한 실수**:
- cv 미설정으로 데이터 누수
- SVM에 probability=True 미설정

**실무 팁**:
- Stacking은 Kaggle 대회에서 자주 사용
- 학습 시간이 오래 걸리므로 최종 단계에서 사용

---

## Q10. 종합: 최적 분류 모델 파이프라인 ⭐⭐⭐⭐⭐

**문제**: 사기 탐지 데이터에 대해 최적의 분류 파이프라인을 구축하세요.

In [ ]:
# 정답 코드 - Part 1: 모델 훈련 및 비교

# 데이터 준비 (앞서 생성한 사기 탐지 데이터 사용)
print("=" * 60)
print("사기 탐지 최적 분류 파이프라인 구축")
print("=" * 60)

# 모델 정의 (불균형 처리 적용)
models = {
    'LR (balanced)': LogisticRegression(class_weight='balanced', random_state=42, max_iter=1000),
    'RF (balanced)': RandomForestClassifier(n_estimators=100, class_weight='balanced', random_state=42, n_jobs=-1),
    'GB': GradientBoostingClassifier(n_estimators=100, random_state=42)
}

# 결과 저장
all_results = []
all_probas = {}

for name, model in models.items():
    # 스케일링 필요 여부 판단
    if 'LR' in name:
        model.fit(X_train_scaled, y_train_f)
        y_pred = model.predict(X_test_scaled)
        y_proba = model.predict_proba(X_test_scaled)[:, 1]
    else:
        model.fit(X_train_f, y_train_f)
        y_pred = model.predict(X_test_f)
        y_proba = model.predict_proba(X_test_f)[:, 1]
    
    all_probas[name] = y_proba
    
    all_results.append({
        'Model': name,
        'Accuracy': accuracy_score(y_test_f, y_pred),
        'Precision': precision_score(y_test_f, y_pred),
        'Recall': recall_score(y_test_f, y_pred),
        'F1': f1_score(y_test_f, y_pred),
        'AUC': auc(*roc_curve(y_test_f, y_proba)[:2]),
        'AP': average_precision_score(y_test_f, y_proba)
    })

# 결과 테이블
df_results = pd.DataFrame(all_results).sort_values('F1', ascending=False)
print("\n[모델 성능 비교]")
print(df_results.round(4).to_string(index=False))

In [ ]:
# 정답 코드 - Part 2: ROC 곡선 시각화

fig = go.Figure()

colors = {'LR (balanced)': 'blue', 'RF (balanced)': 'green', 'GB': 'red'}

for name, y_proba in all_probas.items():
    fpr, tpr, _ = roc_curve(y_test_f, y_proba)
    roc_auc = auc(fpr, tpr)
    
    fig.add_trace(go.Scatter(
        x=fpr, y=tpr,
        mode='lines',
        name=f"{name} (AUC={roc_auc:.3f})",
        line=dict(color=colors[name], width=2)
    ))

fig.add_trace(go.Scatter(
    x=[0, 1], y=[0, 1],
    mode='lines',
    name='Random',
    line=dict(color='gray', dash='dash')
))

fig.update_layout(
    title='ROC Curves - 모델 비교',
    xaxis_title='False Positive Rate',
    yaxis_title='True Positive Rate',
    template='plotly_white',
    width=700, height=500
)
fig.show()

In [ ]:
# 정답 코드 - Part 3: PR 곡선 시각화

fig = go.Figure()

for name, y_proba in all_probas.items():
    precision, recall, _ = precision_recall_curve(y_test_f, y_proba)
    ap = average_precision_score(y_test_f, y_proba)
    
    fig.add_trace(go.Scatter(
        x=recall, y=precision,
        mode='lines',
        name=f"{name} (AP={ap:.3f})",
        line=dict(color=colors[name], width=2)
    ))

# 베이스라인
baseline = y_test_f.mean()
fig.add_hline(y=baseline, line_dash="dash", line_color="gray",
              annotation_text=f"Baseline ({baseline:.2%})")

fig.update_layout(
    title='Precision-Recall Curves - 모델 비교',
    xaxis_title='Recall',
    yaxis_title='Precision',
    template='plotly_white',
    width=700, height=500
)
fig.show()

In [ ]:
# 정답 코드 - Part 4: 최종 추천 및 설명

print("\n" + "=" * 60)
print("최종 추천 모델 및 이유")
print("=" * 60)

# 최고 성능 모델 찾기
best_model = df_results.iloc[0]['Model']
best_metrics = df_results.iloc[0]

print(f"\n추천 모델: {best_model}")
print(f"\n성능 지표:")
print(f"  - Accuracy: {best_metrics['Accuracy']:.4f}")
print(f"  - Precision: {best_metrics['Precision']:.4f}")
print(f"  - Recall: {best_metrics['Recall']:.4f}")
print(f"  - F1 Score: {best_metrics['F1']:.4f}")
print(f"  - AUC-ROC: {best_metrics['AUC']:.4f}")
print(f"  - Average Precision: {best_metrics['AP']:.4f}")

print(f"\n선택 이유:")
print(f"  1. F1 Score가 가장 높음 ({best_metrics['F1']:.4f})")
print(f"  2. Recall이 높아 사기 탐지에 적합 ({best_metrics['Recall']:.4f})")
print(f"  3. class_weight='balanced'로 불균형 처리 적용")
print(f"  4. 앙상블로 안정적인 성능")

print(f"\n실무 적용 시 고려사항:")
print(f"  - 임계값 조정으로 Recall/Precision 균형 조절 가능")
print(f"  - 정기적인 재훈련으로 새로운 사기 패턴 학습")
print(f"  - 실시간 모니터링으로 모델 성능 추적")

In [ ]:
# 테스트
assert len(all_results) >= 3, "최소 3개 모델 비교가 필요합니다"
assert df_results['F1'].max() > 0.5, "최고 F1이 0.5 이상이어야 합니다"
print("테스트 통과!")

### 풀이 설명

**접근 방법**:
1. 불균형 처리 적용한 모델 3개 이상 훈련
2. 다양한 지표로 성능 평가
3. ROC, PR 곡선으로 시각화
4. 비즈니스 관점에서 최적 모델 선택

**핵심 개념**:
- 사기 탐지는 불균형 문제 -> Recall 중시
- class_weight로 불균형 처리
- PR 곡선이 ROC보다 불균형 데이터에 유용
- F1 Score로 Precision-Recall 균형 평가

**대안**:
- SMOTE 오버샘플링 적용
- XGBoost/LightGBM 사용
- 앙상블(Voting, Stacking) 적용

**흔한 실수**:
- Accuracy만으로 평가 (불균형에서 부적합)
- 불균형 처리 미적용
- 단일 지표만 고려

**실무 팁**:
- 사기 탐지: FN 비용 > FP 비용 -> Recall 우선
- 임계값 조정으로 운영 환경에 맞게 조절
- A/B 테스트로 실제 비즈니스 영향 측정
- 모델 해석(SHAP, 특성 중요도)으로 신뢰성 확보

---

## 학습 정리

### 퀴즈 난이도 분포

| 난이도 | 퀴즈 번호 | 핵심 개념 |
|--------|-----------|----------|
| ⭐ | Q1, Q2 | 기본 모델 훈련, KNN K값 실험 |
| ⭐⭐ | Q3, Q4, Q5 | 특성 중요도, 모델 비교, Confusion Matrix |
| ⭐⭐⭐ | Q6, Q7, Q8 | Voting, class_weight, ROC 시각화 |
| ⭐⭐⭐⭐ | Q9 | Stacking Ensemble |
| ⭐⭐⭐⭐⭐ | Q10 | 종합 파이프라인 |

### 핵심 학습 포인트

1. **기본 분류기**: LR, KNN, NB, SVM - 각각의 장단점 이해
2. **트리 기반**: DT, RF, GB - 특성 중요도 활용
3. **앙상블**: Voting, Bagging, Boosting, Stacking - 성능 극대화
4. **불균형 처리**: class_weight, SMOTE - 실무 필수
5. **시각화**: ROC, PR 곡선 - Plotly로 인터랙티브

### 다음 단계

- Day 14-0: 군집화 (Clustering)
- Day 14-1: 차원 축소 (Dimensionality Reduction)
- Week 4 Kaggle 미니 대회 준비